In [62]:
using Pkg
Pkg.activate(pwd())
using QuantumDots
using LinearAlgebra
using Plots
using Symbolics
using Latexify
using QuantumDots.BlockDiagonals
const N = 2

  Activating project at `C:\Users\vi2823sv\.julia\dev\QuantumDots\examples`


2

In [63]:
c = FermionBasis(1:N; qn = QuantumDots.parity)
kitaev_hamiltonian(c,μ,t,Δ,V) = blockdiagonal(sum(μ*c[i]'c[i] for i in 1:N) + 
sum(t*c[i+1]'*c[i] + Δ*c[i+1]*c[i] + QuantumDots.hc + V*c[i]'c[i]*c[i+1]'*c[i+1] for i in 1:N-1),c)

kitaev_hamiltonian (generic function with 1 method)

In [64]:
@variables μ t Δ V
H = kitaev_hamiltonian(c, μ, t, Δ, V)

4×4 BlockDiagonal{Num, SparseArrays.SparseMatrixCSC{Num, Int64}}:
 μ  t   0       0
 t  μ   0       0
 0  0   0      -Δ
 0  0  -Δ  V + 2μ

In [87]:
@variables μ t Δ V

4-element Vector{Num}:
 μ
 t
 Δ
 V

In [65]:
print(latexify(H))

\begin{equation}
\left[
\begin{array}{cccc}
\mu & t & 0 & 0 \\
t & \mu & 0 & 0 \\
0 & 0 & 0 &  - \Delta \\
0 & 0 &  - \Delta & V + 2 \mu \\
\end{array}
\right]
\end{equation}


BlockDiagonals.BlockDiagonal{Num, SparseArrays.SparseMatrixCSC{Num, Int64}}

In [66]:
diagH = QuantumDots.diagonalize(kitaev_hamiltonian(c,1.0,2,3,0))

QuantumDots.DiagonalizedHamiltonian{Vector{Float64}, BlockDiagonal{Float64, Matrix{Float64}}}([-1.0, 3.0, -2.162277660168379, 4.16227766016838], [-0.7071067811865475 0.7071067811865475 0.0 0.0; 0.7071067811865475 0.7071067811865475 0.0 0.0; 0.0 0.0 -0.8112421851755609 -0.584710284663765; 0.0 0.0 -0.584710284663765 0.8112421851755609])

In [82]:
ground_states = [block.vectors[:,1] for block in blocks(diagH; full=true)]

2-element Vector{Vector{Float64}}:
 [-0.7071067811865475, 0.7071067811865475, 0.0, 0.0]
 [0.0, 0.0, -0.8112421851755609, -0.584710284663765]

In [84]:
ground_states[2]

4-element Vector{Float64}:
  0.0
  0.0
 -0.8112421851755609
 -0.584710284663765

In [85]:
function majorana_coeffs(ϕe,ϕo)

1